# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Input-preparation" data-toc-modified-id="Input-preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Input preparation</a></div><div class="lev1 toc-item"><a href="#Create-mosaics" data-toc-modified-id="Create-mosaics-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Create mosaics</a></div><div class="lev2 toc-item"><a href="#Parallel-production" data-toc-modified-id="Parallel-production-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Parallel production</a></div><div class="lev1 toc-item"><a href="#xy2latlon" data-toc-modified-id="xy2latlon-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>xy2latlon</a></div><div class="lev1 toc-item"><a href="#Combining-campt-results" data-toc-modified-id="Combining-campt-results-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Combining campt results</a></div><div class="lev1 toc-item"><a href="#Tile-coordinates" data-toc-modified-id="Tile-coordinates-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Tile coordinates</a></div>

# Input preparation

In [4]:
from hirise_tools.downloads import download_RED_product
from hirise_tools.products import RED_PRODUCT_ID

import logging
from nbtools.logging import setup_live_logging


In [2]:
from planet4 import io

In [3]:
db = io.DBManager()

In [6]:
data = db.get_image_name_markings('ESP_011447_0950')

In [8]:
data[['image_x','image_y']].describe()

,image_x,image_y
count,26607.000000,26607.000000
mean,793.621958,34533.180745
std,425.209154,12739.384970
min,-24.500000,67.000000
25%,440.250000,28559.500000
50%,804.000000,34935.000000
75%,1135.500000,45583.500000
max,1591.000000,59685.000000


In [8]:
from planet4 import region_data
obsids = []
for season in ['season2', 'season3']:
    for region in ['Ithaca']:
        regio = getattr(region_data, region)
        season_list = getattr(regio, season)
        obsids.extend(season_list)


In [9]:
obsids

['ESP_011350_0945',
 'ESP_011351_0945',
 'ESP_011403_0945',
 'ESP_011404_0945',
 'ESP_011931_0945',
 'ESP_012063_0945',
 'ESP_012076_0945',
 'ESP_012643_0945',
 'ESP_012854_0945',
 'ESP_012858_0855',
 'ESP_021491_0950',
 'ESP_020779_0950',
 'ESP_020568_0950',
 'ESP_020476_0950',
 'ESP_020357_0950',
 'ESP_020146_0950']

In [10]:
obsids = ['ESP_020568_0950', 'ESP_020779_0950']

In [11]:
setup_live_logging('planet4', logging.DEBUG)

<Logger planet4 (DEBUG)>

# Create mosaics

In [12]:
from planet4.projection import create_RED45_mosaic
from planet4 import io

In [13]:
for obsid in obsids:
    create_RED45_mosaic(obsid)

2017-06-19 19:38:54,724 - planet4.projection - Processing the EDR data associated with ESP_020568_0950


/Volumes/Data/planet4/p4_ground_projection/ESP_020568_0950/ESP_020568_0950_mosaic_RED45.cub already exists and I am not allowed to overwrite.


('ESP_020568_0950', False)

2017-06-19 19:38:55,084 - planet4.projection - Processing the EDR data associated with ESP_020779_0950


/Volumes/Data/planet4/p4_ground_projection/ESP_020779_0950/ESP_020779_0950_mosaic_RED45.cub already exists and I am not allowed to overwrite.


('ESP_020779_0950', False)

## Parallel production

In [9]:
from ipyparallel import Client
c = Client()

lbview = c.load_balanced_view()
dview = c.direct_view()

In [10]:
with c[:].sync_imports():
    from planet4.projection import create_RED45_mosaic

importing create_RED45_mosaic from planet4.projection on engine(s)


In [11]:
results = lbview.map_async(create_RED45_mosaic, obsids)

In [12]:
from nbtools import display_multi_progress

In [13]:
display_multi_progress(results, obsids)

KeyboardInterrupt: 

In [14]:
for res in results:
    print(res)

('ESP_013113_0985', False)
('ESP_013034_0985', False)
('ESP_012889_0985', False)
('ESP_012744_0985', False)
('ESP_012691_0985', False)
('ESP_012467_0985', False)
('ESP_012322_0985', False)
('ESP_012256_0985', False)
('ESP_011900_0985', False)
('ESP_011729_0985', False)
('ESP_011702_0985', False)
('ESP_011623_0985', False)
('ESP_011557_0985', False)
('ESP_011544_0985', False)
('ESP_011491_0985', False)
('ESP_022699_0985', False)
('ESP_021460_0985', False)
('ESP_020959_0985', False)
('ESP_020748_0985', False)
('ESP_020194_0985', False)
('ESP_020128_0985', False)
('ESP_020049_0985', False)
('ESP_021829_0985', False)
('ESP_021684_0985', True)
('ESP_021671_0985', False)
('ESP_021605_0985', False)
('ESP_021526_0985', False)
('ESP_020827_0985', False)
('ESP_020339_0985', False)
('ESP_020115_0985', False)


# xy2latlon

In [6]:
from pysis.isis import campt
from pysis.exceptions import ProcessError
import pvl
from planet4 import io

In [7]:
from pathlib import Path
edrpath = io.get_ground_projection_root()
clusterpath = io.analysis_folder() / 'p4_catalog'

In [8]:
obsid = obsids[0]
obsid

'ESP_011350_0945'

In [9]:
list(clusterpath.glob(f"{obsid}_*.csv"))

[PosixPath('/Users/klay6683/Dropbox/data/planet4/p4_analysis/p4_catalog/ESP_011350_0945_blotches.csv'),
 PosixPath('/Users/klay6683/Dropbox/data/planet4/p4_analysis/p4_catalog/ESP_011350_0945_blotches_base_campt_out.csv'),
 PosixPath('/Users/klay6683/Dropbox/data/planet4/p4_analysis/p4_catalog/ESP_011350_0945_blotches_latlons.csv'),
 PosixPath('/Users/klay6683/Dropbox/data/planet4/p4_analysis/p4_catalog/ESP_011350_0945_fans.csv'),
 PosixPath('/Users/klay6683/Dropbox/data/planet4/p4_analysis/p4_catalog/ESP_011350_0945_fans_base_campt_out.csv')]

In [10]:
fpaths = [item for obsid in obsids for item in (clusterpath.glob("{}_*.csv".format(obsid)))]

In [11]:
from nbtools import execute_in_parallel

In [16]:
from ipyparallel import Client
c = Client()

lbview = c.load_balanced_view()
dview = c.direct_view()

In [17]:
with dview.sync_imports():
    from pysis.isis import campt
    from pysis.exceptions import ProcessError
    from pathlib import Path
    from ipyparallel import CompositeError

In [28]:
def do_campt(mosaicname, savepath, temppath):
    print("Calling do_campt")
    try:
        campt(from_=mosaicname, to=savepath, format='flat', append='no',
              coordlist=temppath, coordtype='image')
    except ProcessError as e:
        print(e.stderr)
        return obsid, False

def obsid_marking_from_clusterpath(inpath):
    """Find obsid and marking kind from HiRISE cluster path.
    
    For example, a cluster path like this:
    '/Users/klay6683/Dropbox/data/planet4/p4_analysis/p4_catalog/ESP_011350_0945_blotches.csv'
    will return:
    ('ESP_011350_0945', 'blotches')
    """
    s = inpath.stem  # stem is 'name' (without folders) minus extension
    # s = ESP_xxxxxx_yyyy_blotches (or ..._fans)
    # obsid = s[:until last '_']
    sep = s.rfind('_')
    return s[:sep], s[sep+1:]

    
class XY2LATLON:
    blotch_coords = ['']  #, 'p1', 'p2', 'p3', 'p4']
    fan_coords = ['']   #, 'arm1', 'arm2']
    coords_switch = dict(blotches=blotch_coords, fans=fan_coords)
    
    def __init__(self, inpath):
        self.inpath = inpath
        self.edrpath = io.get_ground_projection_root()
        
    @property
    def obsid(self):
        return obsid_marking_from_clusterpath(self.inpath)[0]
    
    @property
    def marking(self):
        return obsid_marking_from_clusterpath(self.inpath)[1]
    
    @property
    def mosaicname(self):
        return f"{self.obsid}_mosaic_RED45.cub"
    
    @property
    def mosaicpath(self):
        return self.edrpath / self.obsid / self.mosaicname
    
    @property
    def df(self):
        return pd.read_csv(str(self.inpath))

    def process_inpath(self):
        df = self.df
        for coord in self.coords_switch[self.marking]:
            print("Working on coordinate:", coord)        
            if coord == '':
                name = 'base'
                tempcoords = ['x', 'y']
    #         else:
    #             name = coord
    #             tempcoords = [coord + '_x', coord + '_y']
            temppath = inpath.with_suffix('.tocampt')
            df[tempcoords].to_csv(str(temppath), header=False, index=False)
            print("name", name)
            savename = f"{inpath.stem}_{name}_campt_out.csv"
            print("savename", savename)
            savepath = clusterpath / savename
            try:
                do_campt(mosaicpath, savepath, temppath)
            except:
                return False
        return True


def xy2latlon(inpath):
    d = dict(inpath=inpath)
    ok = process_inpath(inpath, *marking_mosaicpath_from_inpath(inpath))
    d['ok'] = ok
    return d

In [44]:
dview.push(dict(process_inpath=process_inpath,
                do_campt=do_campt,
                blotch_coords=blotch_coords,
                fan_coords=fan_coords,
                clusterpath=clusterpath,
                edrpath=edrpath))

<AsyncResult: _push>

In [48]:
results = lbview.map_async(xy2latlon, fpaths)

In [49]:
from nbtools import display_multi_progress

display_multi_progress(results, fpaths)

In [51]:
res = pd.DataFrame(results.result())

In [52]:
res.ok.value_counts()

True    113
Name: ok, dtype: int64

In [ ]:
res[res.ok==False].inpath.values

# Combining campt results

In [11]:
p = fpaths[0]
p

PosixPath('/Users/klay6683/Dropbox/data/planet4/p4_analysis/p4_catalog/ESP_011350_0945_blotches.csv')

In [14]:
class GroundMarking(object):
    def __init__(self, resultfile):
        self.p = Path(resultfile)
        
        # this loop creates filename paths for all coords campt output files
        # and assigns them to object attributes, like
        # self.basefile, self.p1file, etc.
        self.paths = []
        self.mapped_coords = []
        for coord in self.coords:
            path = self.campt_fname(coord)
            setattr(self, coord+'file', path)
            self.paths.append(path)
            self.store_mapped_coords(coord, path)
        self.mapped_coords = pd.concat(self.mapped_coords, axis=1)
        newpath = self.p.with_name(self.p.stem+'_latlons.csv')
        self.mapped_coords.to_csv(str(newpath), index=False)
        self.coordspath = newpath

    def campt_fname(self, coordname):
        return self.p.with_name(self.p.stem + '_{}_campt_out.csv'.format(coordname))
    
    def store_mapped_coords(self, coord, path):
        df = pd.read_csv(path)
        subdf = df[['PlanetographicLatitude',
                    'PositiveEast360Longitude']]
        subdf.columns = [coord+'_lat', coord+'_lon']
        self.mapped_coords.append(subdf)

class GroundBlotch(GroundMarking):
    coords = ['base']  #, 'p1', 'p2', 'p3', 'p4']
    kind = 'blotch'


class GroundFan(GroundMarking):
    coords = ['base', 'arm1', 'arm2']
    kind = 'fan'

    
def get_ground_marking(fname):
    tokens = Path(fname).stem.split('_')
    if tokens[-1] == 'blotches':
        return GroundBlotch(fname)
    else:
        return GroundFan(fname)

In [16]:
p

PosixPath('/Users/klay6683/Dropbox/data/planet4/p4_analysis/p4_catalog/ESP_011350_0945_blotches.csv')

In [15]:
for path in fpaths:
    print(path.stem)
    get_ground_marking(path)

ESP_011350_0945_blotches


ESP_011350_0945_blotches_base_campt_out


FileNotFoundError: File b'/Users/klay6683/Dropbox/data/planet4/p4_analysis/p4_catalog/ESP_011350_0945_blotches_base_campt_out_base_campt_out.csv' does not exist

# Tile coordinates

In [36]:
from planet4.projection import TileCalculator, xy_to_hirise
from planet4 import projection as proj

In [37]:
edrpath

PosixPath('/Volumes/Data/planet4/p4_ground_projection')

In [38]:
cubepaths = [edrpath / obsid / f"{obsid}_mosaic_RED45.cub" for obsid in obsids]

In [39]:
cubepaths

[PosixPath('/Volumes/Data/planet4/p4_ground_projection/ESP_011350_0945/ESP_011350_0945_mosaic_RED45.cub'),
 PosixPath('/Volumes/Data/planet4/p4_ground_projection/ESP_011351_0945/ESP_011351_0945_mosaic_RED45.cub'),
 PosixPath('/Volumes/Data/planet4/p4_ground_projection/ESP_011403_0945/ESP_011403_0945_mosaic_RED45.cub'),
 PosixPath('/Volumes/Data/planet4/p4_ground_projection/ESP_011404_0945/ESP_011404_0945_mosaic_RED45.cub'),
 PosixPath('/Volumes/Data/planet4/p4_ground_projection/ESP_011931_0945/ESP_011931_0945_mosaic_RED45.cub'),
 PosixPath('/Volumes/Data/planet4/p4_ground_projection/ESP_012063_0945/ESP_012063_0945_mosaic_RED45.cub'),
 PosixPath('/Volumes/Data/planet4/p4_ground_projection/ESP_012076_0945/ESP_012076_0945_mosaic_RED45.cub'),
 PosixPath('/Volumes/Data/planet4/p4_ground_projection/ESP_012643_0945/ESP_012643_0945_mosaic_RED45.cub'),
 PosixPath('/Volumes/Data/planet4/p4_ground_projection/ESP_012854_0945/ESP_012854_0945_mosaic_RED45.cub'),
 PosixPath('/Volumes/Data/planet4/p4_

In [40]:
for cubepath in cubepaths:
    print(cubepath)
    tilecalc = TileCalculator(cubepath)
    tilecalc.calc_tile_coords()

/Volumes/Data/planet4/p4_ground_projection/ESP_011350_0945/ESP_011350_0945_mosaic_RED45.cub
Calling do_campt
Created /Volumes/Data/planet4/p4_ground_projection/ESP_011350_0945/ESP_011350_0945_tile_coords.csv
/Volumes/Data/planet4/p4_ground_projection/ESP_011351_0945/ESP_011351_0945_mosaic_RED45.cub
Calling do_campt
Created /Volumes/Data/planet4/p4_ground_projection/ESP_011351_0945/ESP_011351_0945_tile_coords.csv
/Volumes/Data/planet4/p4_ground_projection/ESP_011403_0945/ESP_011403_0945_mosaic_RED45.cub
Calling do_campt
Created /Volumes/Data/planet4/p4_ground_projection/ESP_011403_0945/ESP_011403_0945_tile_coords.csv
/Volumes/Data/planet4/p4_ground_projection/ESP_011404_0945/ESP_011404_0945_mosaic_RED45.cub
Calling do_campt
Created /Volumes/Data/planet4/p4_ground_projection/ESP_011404_0945/ESP_011404_0945_tile_coords.csv
/Volumes/Data/planet4/p4_ground_projection/ESP_011931_0945/ESP_011931_0945_mosaic_RED45.cub
Calling do_campt
Created /Volumes/Data/planet4/p4_ground_projection/ESP_0119

In [30]:
bucket = []
for cubepath in cubepaths:
    tilecalc = TileCalculator(cubepath)
    img_name = tilecalc.img_name
    final_fname = f"{img_name}_tile_coords.csv"
    coordspath = Path(cubepath).parent / final_fname
    df = pd.read_csv(coordspath)
    df['image_name'] = img_name
    bucket.append(df)

In [31]:
coords = pd.concat(bucket, ignore_index=True)

In [32]:
coords.head()

,xy_hirise,x_hirise,y_hirise,PlanetocentricLatitude,PlanetographicLatitude,PositiveEast360Longitude,image_id,x_tile,y_tile,image_name
0,"(420.0, 324.0)",420.0,324.0,-81.483714,-81.582232,296.098134,APF000014i,1,1,ESP_013113_0985
1,"(420.0, 872.0)",420.0,872.0,-81.479361,-81.577928,296.089005,APF0000118,1,2,ESP_013113_0985
2,"(420.0, 1420.0)",420.0,1420.0,-81.475006,-81.573622,296.079898,APF000013o,1,3,ESP_013113_0985
3,"(420.0, 1968.0)",420.0,1968.0,-81.470644,-81.569309,296.070956,APF000010q,1,4,ESP_013113_0985
4,"(420.0, 2516.0)",420.0,2516.0,-81.466281,-81.564995,296.061811,APF000014f,1,5,ESP_013113_0985


In [33]:
coords.to_csv('/Users/klay6683/Dropbox/data/planet4/p4_analysis/for_chase/inca_tile_coords.csv')

In [214]:
meta = pd.read_hdf('/Users/klay6683/Dropbox/SternchenAndMe/python_stuff/hirise_rdr_index.hdf')